<a href="https://colab.research.google.com/github/aslyldrm/newyork_weather_air_quality_web/blob/main/Newyork_Air_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('updated_air_quality_data.csv')

dataset.drop(columns=['Unique ID','Start_Date'], inplace=True)






## Encoding categorical data

In [3]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), [0,1,2,3,4])], remainder='passthrough')
# X = np.array(ct.fit_transform(X))

# ['Emissions' 'General Pollution' 'Asthma-Related' 'Hospitalizations']
air_quality_map = {'Emissions': 0, 'General Pollution': 1, 'Asthma-Related': 2, 'Hospitalizations': 3}
dataset['Name'] = dataset['Name'].map(air_quality_map)

#['Density' 'Miles' 'Rates' 'Concentration']
air_quality_map = {'Density': 0, 'Miles': 1, 'Rates': 2, 'Concentration':3}
dataset['Measure'] = dataset['Measure'].map(air_quality_map)

#['UHF42' 'UHF34' 'Borough' 'CD' 'Citywide']
air_quality_map = {'UHF42': 0, 'UHF34': 1, 'Borough': 2, 'CD':3,'Citywide':4}
dataset['Geo Type Name'] = dataset['Geo Type Name'].map(air_quality_map)

#['Queens' 'Unknown' 'Brooklyn' 'Bronx' 'Manhattan' 'Staten Island']
air_quality_map = {'Queens': 0, 'Unknown': 1, 'Brooklyn': 2, 'Bronx':3,'Manhattan':4,'Staten Island':5}
dataset['Geo Place Name'] = dataset['Geo Place Name'].map(air_quality_map)

# ['Other' 'Annual Average' 'Summer' 'Winter']
air_quality_map = {'Other':0, 'Annual Average':1, 'Summer':2, 'Winter':3}
dataset['Time Period'] = dataset['Time Period'].map(air_quality_map)

air_quality_map = {'Bad': 0, 'Moderate': 1, 'Good': 2}
dataset['Air Quality Category'] = dataset['Air Quality Category'].map(air_quality_map)

In [4]:
dataset

,Name,Measure,Geo Type Name,Geo Place Name,Time Period,Data Value,Air Quality Category
0,0,0,0,0,0,0.3,2
1,0,0,0,1,0,1.2,2
2,1,1,0,1,1,8.6,2
3,1,1,0,0,1,8.0,2
4,1,1,0,0,2,6.1,2
...,...,...,...,...,...,...,...
18011,1,1,3,1,0,12.9,1
18012,1,1,3,1,0,14.7,1
18013,1,1,0,1,0,43.4,0
18014,1,1,0,0,0,65.8,0


In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

## Splitting the dataset into the Training set and Test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [9]:
# !pip uninstall -y scikit-learn
# !pip install scikit-learn==1.5.2


## Training XGBoost on the Training set

In [10]:
import pickle
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)

In [11]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [12]:
# Vektörleştiriciyi ve modeli pickle ile kaydetme
with open('cv.pkl', 'wb') as f:
    pickle.dump(X_train, f)

with open('clf.pkl', 'wb') as f:
    pickle.dump(classifier, f)

## Making the Confusion Matrix

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1318    0    0]
 [   0 1715    0]
 [   0    0 1471]]


1.0

## Applying k-Fold Cross Validation

In [14]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 99.90 %
Standard Deviation: 0.19 %
